In [31]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("Création de Dataset à partir d'un fichier CSV")
  .getOrCreate()



spark = org.apache.spark.sql.SparkSession@2891bb75


org.apache.spark.sql.SparkSession@2891bb75

In [32]:
case class Song(Artist: String, TrackName: String, TrackId: String, TimesRepeated: Float, acousticness: Float, danceability: Float, energy: Float, instrumentalness: Float, key: Float, liveness: Float, loudness: Float, mode: Float, speechness: Float, tempo: Float, valence: Float, release_date: String)

defined class Song


In [33]:
val customSchema = StructType(Array(
  StructField("Artist",StringType,true),
  StructField("TrackName",StringType,true),
  StructField("TrackId",StringType,true),
  StructField("TimesRepeated", FloatType,true),
    StructField("acousticness", FloatType,true),
    StructField("danceability", FloatType,true),
    StructField("energy", FloatType,true),
    StructField("instrumentalness", FloatType,true),
    StructField("key", FloatType,true),
    StructField("liveness", FloatType,true),
    StructField("loudness", FloatType,true),
    StructField("mode", FloatType,true),
    StructField("speechness", FloatType,true),
    StructField("tempo", FloatType,true),
    StructField("valence", FloatType,true),
    StructField("release_date", StringType,true)

))

customSchema = StructType(StructField(Artist,StringType,true),StructField(TrackName,StringType,true),StructField(TrackId,StringType,true),StructField(TimesRepeated,FloatType,true),StructField(acousticness,FloatType,true),StructField(danceability,FloatType,true),StructField(energy,FloatType,true),StructField(instrumentalness,FloatType,true),StructField(key,FloatType,true),StructField(liveness,FloatType,true),StructField(loudness,FloatType,true),StructField(mode,FloatType,true),StructField(speechness,FloatType,true),StructField(tempo,FloatType,true),StructField(valence,FloatType,true),StructField(release_date,StringType,true))


StructType(StructField(Artist,StringType,true),StructField(TrackName,StringType,true),StructField(TrackId,StringType,true),StructField(TimesRepeated,FloatType,true),StructField(acousticness,FloatType,true),StructField(danceability,FloatType,true),StructField(energy,FloatType,true),StructField(instrumentalness,FloatType,true),StructField(key,FloatType,true),StructField(liveness,FloatType,true),StructField(loudness,FloatType,true),StructField(mode,FloatType,true),StructField(speechness,FloatType,true),StructField(tempo,FloatType,true),StructField(valence,FloatType,true),StructField(release_date,StringType,true))

In [39]:

var resultatFinal: DataFrame = null
val list_country = List("egypt", "morocco","nigeria","southafrica","india","japan","korea","uae","france","germany","greece","italy","sweden","costarica","canada","mexico","usa","australia","newzealand","argentina","brazil","chile","peru")
val list_continent = List("africa", "africa","africa","africa","asia","asia","asia","asia","europe","europe","europe","europe","europe","north_america","north_america","north_america","north_america","ocenia","ocenia","south_america","south_america","south_america","south_america")

// Créez une liste de paires (pays, continent)
val countryContinentPairs = list_country.zip(list_continent)

// Définissez une fonction pour rechercher le continent correspondant pour un pays donné
def findContinent(country: String): String = {
  countryContinentPairs.find { case (c, continent) => c == country }.map(_._2).getOrElse("Continent not found")
}

for (country <- list_country) {
  val continent = findContinent(country)
  

val table = spark.read.format("csv")
  .option("sep", ",")
  .option("header", "true")
  .schema(customSchema)
  .load("charts_data_release_dates/"+"charts_data_"+continent+"/"+country+"_chart_data.csv")

//table.schema
table.as[Song]
val table3 = table.withColumn("date", to_date(table{"release_date"}, "yyyy-MM-dd"))
val table4 = table3.withColumn("year", year(table{"release_date"}))
val table2=table4.withColumn("country",lit(country))
val tableTri = table2.orderBy(desc("TimesRepeated"))
val topsong = table2.limit(10)
//topsong.collect.foreach(println)
    val avgtop= topsong.groupBy("country").agg(avg("TimesRepeated").alias("avg_TimesRepeated"),
                        avg("acousticness").alias("avg_acousticness"),
                        avg("danceability").alias("avg_danceability"),
                        avg("energy").alias("avg_energy"),
                        avg("instrumentalness").alias("avg_instrumentalness"),
                        avg("key").alias("avg_key"),
                        avg("liveness").alias("avg_liveness"),
                        avg("loudness").alias("avg_loudness"),
                        avg("mode").alias("avg_mode"),
                        avg("speechness").alias("avg_speechness"),
                        avg("tempo").alias("avg_tempo"),
                        avg("valence").alias("avg_valence"),avg("year").alias("avg_year")).withColumn("continent",lit(continent))
  //val avgtop2=avgtop.withColumn("continent",lit(continent))  
if (resultatFinal == null) {
    resultatFinal = avgtop
    
  } else {
    resultatFinal = resultatFinal.union(avgtop)
    
  }

}


resultatFinal = [country: string, avg_TimesRepeated: double ... 13 more fields]
list_country = List(egypt, morocco, nigeria, southafrica, india, japan, korea, uae, france, germany, greece, italy, sweden, costarica, canada, mexico, usa, australia, newzealand, argentina, brazil, chile, peru)
list_continent = List(africa, africa, africa, africa, asia, asia, asia, asia, europe, europe, europe, europe, europe, north_america, north_america, north_america, north_america, ocenia, ocenia, south_america, south_america, south_america, south_america)
countryContinentPairs = List((egypt,africa), (morocco,africa), (nigeria,africa), (southafrica,africa), (india,asia), (japan,asia), (korea,asia), (uae,asia), (france,eur...


List((egypt,africa), (morocco,africa), (nigeria,africa), (southafrica,africa), (india,asia), (japan,asia), (korea,asia), (uae,asia), (france,eur...

In [40]:
resultatFinal.write
.format("csv")
  .option("header", "true")// Inclure l'en-tête dans le fichier CSV
 .mode("overwrite")
  .save("result")

In [41]:
val resultcontinent= resultatFinal.groupBy("continent").agg(avg("avg_TimesRepeated").alias("avg_TimesRepeated"),
                        avg("avg_acousticness").alias("avg_acousticness"),
                        avg("avg_danceability").alias("avg_danceability"),
                        avg("avg_energy").alias("avg_energy"),
                        avg("avg_instrumentalness").alias("avg_instrumentalness"),
                        avg("avg_key").alias("avg_key"),
                        avg("avg_liveness").alias("avg_liveness"),
                        avg("avg_loudness").alias("avg_loudness"),
                        avg("avg_mode").alias("avg_mode"),
                        avg("avg_speechness").alias("avg_speechness"),
                        avg("avg_tempo").alias("avg_tempo"),
                        avg("avg_valence").alias("avg_valence"),avg("avg_year").alias("avg_year"))

resultcontinent = [continent: string, avg_TimesRepeated: double ... 12 more fields]


[continent: string, avg_TimesRepeated: double ... 12 more fields]

In [42]:
resultcontinent.write
.format("csv")
  .option("header", "true")  // Inclure l'en-tête dans le fichier CSV
 .mode("overwrite")
  .save("resultcontinent")